In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py

from matplotlib import animation
from matplotlib import cm
from mpl_toolkits import mplot3d
import os,sys
sys.path.append("/Users/ajc/Core/Projects/ATS-Data/OR-CONDO/PyScript")
import readdata as rd
import os
import matplotlib.gridspec as gridspec

In [36]:

mesh_file= "/Users/ajc/Core/Projects/ATS-Data/OR-CONDO/mytests-orhydra/ats-transport-subgrid-testsuite/testing/transport-coweeta/test0/"

mesh_file='/Users/ajc/FUSE/ats-tests/ats-transport-subgrid-testsuite/testing/transport-coweeta/test0/'

infile = [os.path.join(mesh_file,'visdump_surface_data.h5'), os.path.join(mesh_file,'visdump_surface_mesh.h5') ]

mesh = h5py.File(infile[1],'r')
mesh_data = list(mesh['0']['Mesh']['Nodes'])

C_data = rd.GetSurfVarFromVis_All(infile[0],'surface-total_component_concentration',suffix='.cell.Tc-99')
Concentration = C_data['surface-total_component_concentration']
cycles = len(Concentration)
print (len(Concentration))

145


In [3]:
X_mesh = []
Y_mesh = []
Z_mesh = []

for m in mesh_data:#[:30]:
    #print (m)
    X_mesh.append(m[0])
    Y_mesh.append(m[1])
    Z_mesh.append(m[2])
print (len(Concentration))


145


In [ ]:
Discharge_m = [0.9080831404970909, 0.374545887545661, 0.43239644141179195, 0.33837020936405693, 0.025871319142538998, 0.204746066590533, 0.128745291619659, 0.03380683234581899, 0.07169457541389,  0.06670998915332699, 0.031497253670805,  0.36350613999000597,  0.265706376594204,  0.09174899121199499,  0.041822935295661,  0.22223661842756398,  0.17922055278355797,  0.016846145349851998,  0.07235466943430699,  0.061198907756493, 0.019760430292550996, 0.054590237053091996]
Discharge_m = [round(d,4) for d in Discharge_m]
Discharge = [d*100 for d in Discharge_m]
print (max(Discharge))

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('/Users/ajc/Core/SimDataInputs/WorkSpace/ats-88/ats_meshing/examples/coweeta_river-centriod1.xml')
root = tree.getroot()
#print (root.attrib)
Ncells = []
for child in root:
    seg = child.attrib['name']
    if 'segment' in seg:
        for c1 in child.iter():
            if 'number of cells' in c1.attrib['name']:
                #print ('OUT',c1.keys(), c1.attrib['name'],c1.get('value'))
                Ncells.append(int(c1.get('value')))


In [ ]:
Discharge_stream = [[d,]*c for d,c in zip(Discharge,Ncells)]
Discharge_stream = np.concatenate(Discharge_stream)


In [ ]:
%matplotlib tk

#fig = plt.figure()
#ax = fig.gca(projection='3d')

fig, ax = plt.subplots(1,1,figsize=(8,4))
x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]

for k in range(1):
    y1 = y_orig#[i + 0.5*thick[k] for i in y_orig[k]]
    y2 = [i + 0.000001 for i in y1]#[i - 0.5*thick[k] for i in y_orig[k]]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*0

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
    #ran = Discharge_stream
    ran = Concentration[1]
    ran = [r if r>0 else 1e-12 for r in ran]
    sc = ax.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1,norm=matplotlib.colors.LogNorm())
    #,vmin=0,vmax=10) map=plt.cm.get_cmap('prism', 5)) cm.tab10
    
    plt.colorbar(sc, label='Discharge [cm$^3$/sec]',shrink=.96)#,aspect=10)
   
    #ax.set_zlim(0,0.0)
    #ax.set_zticks([])

    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax.get_ylim()
    ax.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax.grid(False)
    plt.tight_layout(h_pad=1)
   
    #plt.savefig('/Users/ajc/Core/PostProcessData/2019/simulations/transport/plots_1002_%s.png'%k,dpi=500)

In [37]:
path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/Coweeta/test0/'

In [38]:
x_orig = X_mesh
y_orig = Y_mesh

centroid = []
thick = [1,1,1]
Time = 0
for k in range(0,cycles,1):
    Time = k*1.0
    print (k)
    #-----------------------------
    y1 = y_orig
    y2 = [i + 0.000001 for i in y1]
    X = [x_orig,x_orig]
    Y = [y1,y2]
    
    z = np.ones(len(x_orig))*0

    Z = np.array([z,z])
    data = np.linspace(1,0,len(x_orig))
    Z_data = np.array([data,data])
    
    data_color = cm.jet(Z_data/np.amax(Z_data))
   
    ran = Concentration[k]
    ran = [r if r>0 else 1e-12 for r in ran]
    #-----------------------------
    
    fig, axs = plt.subplots(1,2,figsize=(12,4),facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=.02)

    ax1 = plt.subplot(121)
    sc = ax1.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-4,vmax=1,norm=matplotlib.colors.LogNorm())
    ax1.set_title('Time %s [hours]'%Time)
    
    cb = plt.colorbar(sc, label='Concentration [kg/m$^3$]',shrink=.96)#,aspect=10)
    cb.set_label('Concentration [kg/m$^3$]',labelpad=-1, rotation=0,y=1.1)
    
    xmin, xmax = ax1.get_xlim()
    ax1.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax1.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax1.get_ylim()
    ax1.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax1.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    ax1.grid(False)
    plt.tight_layout(h_pad=1)
    
    #-----------------------
    ax2 = plt.subplot(122)
    ran = [f*100 for f in flux_interpolate] #Discharge_stream
    [r if r>0 else 1e-12 for r in ran]
    #sc = ax2.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.tab10,vmin=0,vmax=100)
    sc = ax2.scatter(X_mesh,Y_mesh,marker=".",c=ran,cmap=cm.coolwarm,vmin=1e-1,vmax=100,norm=matplotlib.colors.LogNorm())
    
    cb = plt.colorbar(sc, label='Discharge [cm$^3$/sec]',shrink=.96)#,aspect=10)
    cb.set_label('Discharge [cm$^3$/sec]',labelpad=-1, rotation=0,y=1.1)
    xmin, xmax = ax2.get_xlim()
    ax2.set_xticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax2.set_xticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=45, fontsize=10)
    
    xmin, xmax = ax2.get_ylim()
    ax2.set_yticks(np.round(np.linspace(xmin, xmax, 5), 2))
    ax2.set_yticklabels(np.round(np.linspace(xmin, xmax, 5), 2), rotation=0, fontsize=10)
    
    ax2.grid(False)
    plt.tight_layout(h_pad=1)
    
    #plt.show()
    plt.savefig(path + 'figs_%s.png'%k,dpi=100)
    plt.close()
    #break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144


In [34]:
import re, os
path='/Users/ajc/Core/PostProcessData/2019/simulations/transport/Coweeta/test1/'
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

dir_files = os.listdir(path)

Files = sorted_nicely(dir_files)

Files = [f for f in Files if f.endswith('.png')]
#print (len(Files))
#Files = Files[:20] + Files[350:400] + Files[450:]
import cv2
import numpy as np

#print (Files)
img_array = []
for filename in Files:
    filename = path + filename
    
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    


In [35]:
out = cv2.VideoWriter(path + 'test1-coweeta.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [6]:
#Discharge_stream = [[d,]*(c+1) for d,c in zip(Discharge_m,Ncells)]
flux_interpolate = np.loadtxt('/Users/ajc/Core/PostProcessData/2019/simulations/transport/flux_interpolate1.dat')
#Discharge_stream = -np.concatenate(Discharge_stream)
Discharge_stream = [-f for f in flux_interpolate]
cross_section_area = [0.8163, 0.3662, 0.4004, 0.3363, 0.0301, 0.1924, 0.1265, 0.0394, 0.0724, 0.0685, 0.0356, 
                      0.3395, 0.2495, 0.091 , 0.0462, 0.2025, 0.1641, 0.0206, 0.072 , 0.0651, 0.025 , 0.0628]
file='/Users/ajc/Core/PostProcessData/2019/simulations/transport/facefluxes_coweeta.h5'

if os.path.exists(file):
    print (file)
    os.remove(file)
outfile = h5py.File(file,'w')
outfile.create_dataset("time", data=np.array([0.,]))
#cs_area = outfile.create_group('cross-sectional area [m^2]')
flux = outfile.create_group("surface-mass_flux.face.0")

flux.create_dataset("0",data=Discharge_stream)
#cs_area.create_dataset("0",data=cross_section_area)

outfile.close()
#print (len(Discharge_m))
#print (len(Discharge_stream), len(Ncells), sum(Ncells))

/Users/ajc/Core/PostProcessData/2019/simulations/transport/facefluxes_coweeta.h5
